In [1]:
import pandas as pd
import json
from monty.json import MontyDecoder,MontyEncoder
from sklearn.metrics import mean_absolute_error,mean_squared_error
import numpy as np
from matplotlib import pyplot as plt
from pymatgen.io.ase import AseAtomsAdaptor

In [2]:
with open("Ti_data_both.json") as f:
    data_collection=json.loads(f.read(),cls=MontyDecoder)

In [20]:
#get reference energy (omega) #
bd=0
for data in data_collection:
    try:
        if data['metadata']['perturbation']=="icsd" and data['calc']=="final" and data['metadata']['proto']=="omega.vasp":
            bd=data['energy']
    except:
        pass

assert bd!=0
bd=bd/len(data['structure'].species)

In [25]:
#get lists of energies, forces, ase_atoms objects, referenced energies#
energies=[]
forces=[]
ase_atoms=[]
energy_corrected=[]
for data in data_collection:
    energies.append(data['energy'])
    forces.append(data['forces'])
    ase_atoms.append(AseAtomsAdaptor.get_atoms(data['structure']))
    energy_corrected.append(data['energy']-bd*len(data['structure'].species))


In [28]:
#create dataframe and dump#
d={
    'energy':energies,
    'forces':forces,
    'ase_atoms':ase_atoms,
    'energy_corrected':energy_corrected
}
df=pd.DataFrame(data=d)
df.to_pickle('my_data.pckl.gzip', compression='gzip', protocol=4)
# dff = pd.read_pickle("my_data.pckl.gzip", compression="gzip")